In [3]:
import matplotlib.pyplot as plt
import numpy as np
import cvxpy
from math import *


In [13]:
class MPC:
    """Model predictive controller using non linear vehicle kinematic model"""
    def __init__(self,x=[0],y=[0],yaw=[0],v=[0],
                 steering_angle=0,acceleration=0, v_max=80,v_min =-1,
                 max_steering_angle = 0.9, a_max = 1, a_min = -1.5,a_rate_max=1.5, a_rate_min=-3, steer_rate=0.175,
                 lr=1.738,lf=1.105,Q = np.eye(4),R = np.eye(2),R_= np.eye(2),Hp = 8,Hc = 8,ts=0.1,td=0.2):
        #init states
        self.x  = x
        self.y  = y
        self.yaw= yaw
        self.v  = v

        #init input variables or MV
        # controller.delta_f = steering_angle
        # self.a       = acceleration

        #init vehicle parameters
        self.lr = lr
        self.lf = lf
        self.L  = lr + lf

        #init weight matrices
        self.Q = Q
        self.R = R
        self.R_= R_

        #constraints
        self.v_max = v_max
        self.v_min = v_min
        self.delta_f_max = max_steering_angle
        self.a_max = a_max
        self.a_min = a_min
        self.steer_rate = steer_rate
        self.a_rate_max = a_rate_max
        self.a_rate_min = a_rate_min

        #horizon and time
        self.Hp = Hp
        self.Hc = Hc
        self.ts = ts
        self.td = td

        #trajectory
        self.z_ref = self.get_ref_traj("A")
        self.prev_idx = 0
        self.send_prev = 0
        self.prev_accelerations = np.array([0.0] * self.Hp)
        self.prev_deltas = np.array([0.0] * self.Hp)
        self.prev_index = 0
        self.dist = 3.5
        #storing states
        self.states = None
        self.inputs = None



    def get_ref_traj(self,type):
        """ 3 types of trajectories available
        A == A right turn with slow speed
        B == A sinusoidal path with morderate speed
        C == CPG winding track"""
        if type == "A":
            T = 81     #total simulation time
            N_steps = int(T/self.ts)
            v = np.array([0.0]*N_steps)
            # constructing the reference velocity profile
            #for 5s speed is 0
            n = int(5/self.ts)
            v[:n] = 0
            #for next 45s speed is 3m/s
            n1 = int(45/self.ts + n)
            v[n:n1] = 3
            #for next 4s speed gradualy reduces to zero
            n = int(4/self.ts)
            for i in range(n):
                v[n1+i] = v[n1+i-1]-0.75*self.ts
            # for next 4 sec speed is 0
            n1 = int(n1 + n)
            n  = int(3/self.ts +n1)
            v[n1:n] = 0.0
            # for next 12s speed is 6m/s
            n1 = int(12/self.ts + n)
            v[n:n1] = 6
            #for next 10 s the speed delecerates to 0
            n = int(10/self.ts)
            for i in range(n):
                v[n1+i] = v[n1+i-1]-0.6*self.ts
            #for next 1s speed is zero
            n1 = int(n1 + n)
            n  = int(1/self.ts)
            v[n1:n] = 0.0
            # #hence total simulation time is 5+45+4+4+12+10+1 = 81s

            # now based on the refrence velocity we get the x,v,yaw of the vehicle
            x   = np.array([0.0]*N_steps)
            y   = np.array([0.0]*N_steps)
            yaw = np.array([0.0]*N_steps)

            #first 30 secs straight road in x direction
            for i in range(int(30/self.ts)-1):
                yaw[i+1] = yaw[i]
                x[i+1] = x[i] + v[i]*cos(yaw[i])*self.ts
                y[i+1] = y[i] + v[i]*sin(yaw[i])*self.ts 
            # turining right in 10s 
            for i in range(int(30/self.ts)-1,int(40/self.ts)-1):
                yaw[i+1] = yaw[i] + pi/20*self.ts
                x[i+1] = x[i] + v[i]*cos(yaw[i])*self.ts
                y[i+1] = y[i] + v[i]*sin(yaw[i])*self.ts
            #again straight for next 31 secs in y direction
            for i in range(int(40/self.ts)-1,int(81/self.ts)-1):
                yaw[i+1] = yaw[i]
                x[i+1] = x[i] + v[i]*cos(yaw[i])*self.ts
                y[i+1] = y[i] + v[i]*sin(yaw[i])*self.ts
        return np.array([x,y,yaw,v])

    def Kinematic_model(self,yaw,steering_angle=0,dt=0.1):
        """Prediction module for vehicle states"""
        # beta = atan(self.lr*tan(steering_angle)/self.L)
        beta = steering_angle
        A = np.array([[1,0,0,cos(yaw+beta)*dt],
                      [0,1,0,sin(yaw+beta)*dt],
                      [0,0,1,sin(beta)*dt/self.lr],
                      [0,0,0,1]])
        
        B = np.array([[0,0],
                     [0,0],
                     [0,0],
                     [0,1*dt]])
        
        return A ,B

    def MPC(self,z_initial):
        """MPC solver """
        z = cvxpy.Variable((4,self.Hp),"z")
        u = cvxpy.Variable((2,self.Hp),"u")
        z_initial = np.array([self.x,self.y,self.yaw,self.v])
        cost = 0
        constraints = [z[:,0] == z_initial.flatten()]
        # constraints = []
        # self.Hp = min(self.Hp,self.z_ref.shape[1])
        if self.z_ref.shape[1] < self.Hp:
            return None, None
        for i in range(self.Hp-1):
            if i != 0:
                cost += cvxpy.quad_form(self.z_ref[:,i] - z[:,i], self.Q)
                cost += cvxpy.quad_form(u[:,i]-u[:,i-1], self.R)
            else:
                u_prev = [self.prev_deltas[0],self.prev_accelerations[0]]
                cost += cvxpy.quad_form(u[:, i] - u_prev, self.R)
            
            cost += cvxpy.quad_form(u[:,i],self.R_)
            
            #constrains

            A,B = self.Kinematic_model(self.z_ref[2,i],self.prev_deltas[np.min([i+1,len(self.prev_deltas)-1])],self.td)
            constraints += [z[:,i+1] == A @ z[:,i] + B @ u[:,i]]
            print(A)
            print(B)

            #velocity limits
            # constraints += [z[3, i] <= self.v_max]
            # constraints += [z[3, i] >= self.v_min]

            #input limits
            constraints += [self.a_min <= u[1, i]]
            constraints += [u[1, i] <= self.a_max]
            constraints += [u[0, i] <= self.delta_f_max]
            constraints += [u[0, i] >= -self.delta_f_max]

            #rate constraints
            if i != 0:
                constraints += [(u[1, i] - u[1, i-1])/self.td <= self.a_rate_max]
                constraints += [(u[1, i] - u[1, i-1])/self.td >= -self.a_rate_max]
                constraints += [(u[0, i] - u[0, i-1])/self.td <= self.steer_rate]
                constraints += [(u[0, i] - u[0, i-1])/self.td >= -self.steer_rate]
            # else:
            #     constraints += [(u[1, i] - self.prev_accelerations[0])/self.ts  <= self.a_rate_max]
            #     constraints += [(u[1, i] - self.prev_accelerations[0]/self.ts ) >= -self.a_rate_max]
            #     constraints += [(u[0, i] - self.prev_deltas[0])/self.ts <= self.steer_rate]
            #     constraints += [(u[0, i] - self.prev_deltas[0])/self.ts >= -self.steer_rate]
        #Quad program

        qp = cvxpy.Problem(cvxpy.Minimize(cost), constraints)
        qp.solve(solver=cvxpy.ECOS_BB, verbose=False)
        print(qp.status)
        if qp.status == cvxpy.OPTIMAL or qp.status == cvxpy.OPTIMAL_INACCURATE:
            x = np.array(z.value[0, :]).flatten()
            print(x)
            y = np.array(z.value[1, :]).flatten()
            print(y)
            v = np.array(z.value[3, :]).flatten()
            print(v)
            yaw = np.array(z.value[2, :]).flatten()
            print(yaw)
            a = np.array(u.value[1, :]).flatten()
            print(a)
            delta = np.array(u.value[0, :]).flatten()
            print(delta)
        else:
            a, delta = None, None

        return a, delta


    def solve(self,x,y,yaw,v):
        """function to call mpc solver"""
        z_intital = np.array([x,y,yaw,v])
        a,delta = self.MPC(z_intital)
        if a is None:
            self.prev_accelerations = self.prev_accelerations[1:]
            self.prev_deltas = self.prev_deltas[1:]
        else:
            self.prev_accelerations = a
            self.prev_deltas = delta
        return self.prev_accelerations[0], self.prev_deltas[0]

    def run_step(self):
        """runs for 1 trajectory"""
        # A,B = self.Kinematic_model(0,0,0)
        # z_initial = A @ np.array([self.x,self.y,self.yaw,self.v]) + B @ np.array([[0],[0]])
        z_initial = [self.x,self.y,self.yaw,self.v]
        self.states = z_initial
        self.inputs = np.array([[0],[0]])
        #step 1
        a,delta = self.solve(z_initial[0],z_initial[1],z_initial[2],z_initial[3])
        for i in range(0,int(81/self.ts)-1):
            print(i)
            print(self.z_ref[2][0]-self.yaw)
            try:
                self.z_ref = np.delete(self.z_ref,0,axis=1)
            except IndexError:
                pass

            A,B = self.Kinematic_model(self.yaw[0],delta,self.ts)
            z = A @ np.array([self.x,self.y,self.yaw,self.v]) + B @ np.array([[delta],[a]])
            self.x = z[0]
            self.y = z[1]
            self.yaw = z[2]
            self.v = z[3]
            a,delta = self.solve(z[0],z[1],z[2],z[3])
            # print(a,delta)
            self.states = np.append(self.states,z,axis=-1)
            self.inputs = np.append(self.inputs,np.array([[delta],[a]]),axis=-1)

        return self.states ,self.inputs



In [14]:
Q = np.array([[  3.5,  0,  0,  0],
                  [  0,  3.5,  0,  0],
                  [  0,  0,  25,  0],
                  [  0,  0,    0,  80]])

    
controller = MPC(Q=Q)

In [15]:
controller.z_ref

array([[  0.        ,   0.        ,   0.        , ...,  93.94820047,
         93.94820047,  93.94820047],
       [  0.        ,   0.        ,   0.        , ..., 156.79820047,
        156.79820047, 156.79820047],
       [  0.        ,   0.        ,   0.        , ...,   1.57079633,
          1.57079633,   1.57079633],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]])

In [16]:
"""runs for 1 trajectory"""
# A,B = self.Kinematic_model(0,0,0)
# z_initial = A @ np.array([self.x,self.y,self.yaw,self.v]) + B @ np.array([[0],[0]])
z_initial = [controller.x,controller.y,controller.yaw,controller.v]
controller.states = z_initial
controller.inputs = np.array([[0],[0]])
#step 1
a,delta = controller.solve(z_initial[0],z_initial[1],z_initial[2],z_initial[3])


[[1.  0.  0.  0.2]
 [0.  1.  0.  0. ]
 [0.  0.  1.  0. ]
 [0.  0.  0.  1. ]]
[[0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0.2]]
[[1.  0.  0.  0.2]
 [0.  1.  0.  0. ]
 [0.  0.  1.  0. ]
 [0.  0.  0.  1. ]]
[[0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0.2]]
[[1.  0.  0.  0.2]
 [0.  1.  0.  0. ]
 [0.  0.  1.  0. ]
 [0.  0.  0.  1. ]]
[[0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0.2]]
[[1.  0.  0.  0.2]
 [0.  1.  0.  0. ]
 [0.  0.  1.  0. ]
 [0.  0.  0.  1. ]]
[[0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0.2]]
[[1.  0.  0.  0.2]
 [0.  1.  0.  0. ]
 [0.  0.  1.  0. ]
 [0.  0.  0.  1. ]]
[[0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0.2]]
[[1.  0.  0.  0.2]
 [0.  1.  0.  0. ]
 [0.  0.  1.  0. ]
 [0.  0.  0.  1. ]]
[[0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0.2]]
[[1.  0.  0.  0.2]
 [0.  1.  0.  0. ]
 [0.  0.  1.  0. ]
 [0.  0.  0.  1. ]]
[[0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0.2]]
optimal
[ 1.16374935e-13 -2.98106490e-14  4.47614449e-09  9.64981578e-09
  1.33919666e-08  1.26281316e-08  1.26267939e-08  8.42936437e-09]
[0. 0. 

In [17]:
a

1.1191469195319696e-07

In [ ]:
for i in range(0,int(81/controller.ts)-1):
    print(i)
    # print(controller.z_ref[2][0]-controller.yaw)
    try:
        controller.z_ref = np.delete(controller.z_ref,0,axis=1)
    except IndexError:
        pass

    A,B = controller.Kinematic_model(controller.yaw[0],delta,controller.ts)
    z = A @ np.array([controller.x,controller.y,controller.yaw,controller.v]) + B @ np.array([[delta],[a]])
    controller.x = z[0]
    controller.y = z[1]
    controller.yaw = z[2]
    controller.v = z[3]
    a,delta = controller.solve(z[0],z[1],z[2],z[3])
    # print(a,delta)
    controller.states = np.append(controller.states,z,axis=-1)
    controller.inputs = np.append(controller.inputs,np.array([[delta],[a]]),axis=-1)



In [1]:
speed = 13.2285009239462026
x = -16.842267990112305
y = -223.22581481933594
yaw = 1.5481880865109414
wp = [[-16.842267990112305, -223.22581481933594, 1.2295178873288235, 10.0], [-14.934813757798112, -217.56050297954596, 1.2788849523922121, 8.0], [-13.570800910472931, -211.65647296667348, 1.4232967392734457, 5.0], [-13.389941634925464, -205.589677844538, 1.6850661456339946, 4.0], [-15.14255852407123, -199.9906652815648, 2.051803250627153, 2.0], [-18.50294038007302, -194.93976996692965, 2.2261284040873046, 0.0]]

prev_deltas = [-0.24681479, -0.40674397, -0.24680603, -0.12345067,  0.        ]
prev_accelerations = [1.99999931, 1.94880953, 1.4488118,  0.94881729, 0.        ]


In [8]:
waypoints[2:1].flatten()

array([], dtype=float64)

In [4]:
waypoints = np.array(wp).T

In [26]:
x0 = np.array([[x], [y], [yaw], [speed]])

In [27]:
x=[0]
y=[0]
yaw=[0]
v=[0]
steering_angle=0
acceleration=0
v_max=80
v_min =-1
max_steering_angle = 0.9
a_max = 1
a_min = -1.5
a_rate_max=1.5
a_rate_min=-3
steer_rate=0.175
lr=1.738
lf=1.105
Q = np.array([[  3.5,  0,  0,  0],
                  [  0,  3.5,  0,  0],
                  [  0,  0,  25,  0],
                  [  0,  0,    0,  80]])
R = np.eye(2)
R_= np.eye(2)
Hp = 6
Hc = 6
ts=0.1
td=0.2
L =lr + lf

In [28]:
v_max = v_max
v_min = v_min
delta_f_max = max_steering_angle
a_max = a_max
a_min = a_min
steer_rate = steer_rate
a_rate_max = a_rate_max
a_rate_min = a_rate_min

In [29]:
z_ref = waypoints

In [30]:
z_initial = x0

In [31]:
import math

In [52]:
def Kinematic_model(self,yaw,steering_angle=0,dt=0.1):
    """Prediction module for vehicle states"""
    # beta = atan(self.lr*tan(steering_angle)/self.L)
    beta = steering_angle
    A = np.array([[1,0,0,cos(yaw+beta)*dt],
                    [0,1,0,sin(yaw+beta)*dt],
                    [0,0,1,sin(beta)*dt/lr],
                    [0,0,0,1]])
    
    B = np.array([[0,0],
                    [0,0],
                    [0,0],
                    [0,1*dt]])
    
    return A ,B


In [63]:
def Kinematic_model(self,yaw, v, steering_angle=0,dt=0.1):
    """Prediction module for vehicle states"""
    # beta = atan(self.lr*tan(steering_angle)/self.L)
    beta = steering_angle
    A = np.array([[1,0, np.cos(yaw)*dt, -v*np.sin(yaw)*dt],
                    [0,1, sin(yaw)*dt,v*np.cos(yaw)*dt],
                    [0,0,1,0],
                    [0,0,dt*np.tan(steering_angle)/L,1]])
    
    B = np.array([[0,0],
                    [0,0],
                    [dt,0],
                    [0,dt*v*(np.tan(steering_angle)**2+1)/L]])
    
    return A ,B


In [33]:
import math

In [18]:
np.zeros((4, 2))

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [171]:
def get_linearized_dynamics(self, yaw, delta, v, dt=0.01):


    A = np.eye(4)
    A[0, 0] = 0
    A[0, 1] = 0
    A[0, 2] = np.cos(yaw + math.atanh((lr*math.tan(delta))/L)) * dt
    A[0, 3] = -v * np.sin(yaw + math.atanh((lr*math.tan(delta))/L)) * dt
    
    A[1, 0] = 0
    A[1, 1] = 0
    A[1, 2] = np.sin(yaw + math.atanh((lr*math.tan(delta))/L)) * dt
    A[1, 3] = v * np.cos(yaw + math.atanh((lr*math.tan(delta))/L)) * dt


    A[2, 0] = 0
    A[2, 1] = 0
    A[2, 2] = 0
    A[2, 3] = 0

    A[3, 0] = 0
    A[3, 1] = 0
    A[3, 2] = math.tan(delta)/(L*np.sqrt((lr**2*math.tan(delta)**2)/L**2 + 1)) * dt
    A[3, 3] = 0


    B = np.zeros((4, 2))
    B[0, 1] =  -(lr*v*np.sin(yaw + math.atan((lr*math.tan(delta))/L))*(math.tan(delta)**2 + 1))/(L*((lr**2*math.tan(delta)**2)/L**2 + 1)) * dt
    B[1, 1] = (lr*v*np.cos(yaw + math.atan((lr*math.tan(delta))/L))*(math.tan(delta)**2 + 1))/(L*((lr**2*math.tan(delta)**2)/L**2 + 1))* dt
    B[2, 0] = 1* dt
    B[3, 1] = (v*(math.tan(delta)**2 + 1))/(L*((lr**2*math.tan(delta)**2)/L**2 + 1)**(1/2)) - (lr**2*v*math.tan(delta)**2*(math.tan(delta)**2 + 1))/(L**3*((lr**2*math.tan(delta)**2)/L**2 + 1)**(3/2))* dt

  

    # tandelta = math.tan(delta)
    # angel = yaw + math.atanh((lr*tandelta)/L)
    # deno1 = np.tan(delta)**2 + 1
    # deno2 = (lr**2*tandelta**2)/L**2 + 1
    # deno3 = L * np.sqrt(deno2)
    # # print(f"angles {yaw}, {angel}, {tandelta}")

    # A = np.array([[ 0, 0, np.cos(angel), -v*np.sin(angel)],
    #             [ 0, 0, np.sin(angel),  v*np.cos(angel)],
    #             [ 0, 0, 0, 0],
    #             [ 0, 0, tandelta/deno3, 0]]) * dt
    # # A = A + np.eye(4)

    # B = np.array([[ 0, -(lr*v*np.sin(angel)*(deno1))/(L*(deno2))],
    #             [ 0, (lr*v*np.cos(angel)*(deno1))/(L*(deno2))],
    #             [ 1, 0],
    #             [ 0, (v*(deno1))/deno3 - (lr**2*v*tandelta**2*(deno1))/(deno3**3)]])
    # B *= dt

    return A, B

In [79]:
Hp

6

In [64]:
"""MPC solver """
z = cvxpy.Variable((4,Hp),"z")
u = cvxpy.Variable((2,Hp),"u")
# z_initial = np.array([x,y,yaw,v])
cost = 0
constraints = [z[:,0] == z_initial.flatten()]

In [65]:
prev_deltas[np.min([ i + 1, len(prev_deltas) - 1])]

0.0

In [66]:
if z_ref.shape[1] < Hp:
    print('NONE')

for i in range(Hp-1):
    if i != 0:
        cost += cvxpy.quad_form(z_ref[:,i] - z[:,i], Q)
        cost += cvxpy.quad_form(u[:,i]-u[:,i-1], R)
    else:
        u_prev = [prev_deltas[0],prev_accelerations[0]]
        cost += cvxpy.quad_form(u[:, i] - u_prev, R)
    
    cost += cvxpy.quad_form(u[:,i],R_)
    
    #constrains

    A,B = Kinematic_model(z_ref[2, i], z_ref[3, i],prev_deltas[np.min([i+1,len(prev_deltas)-1])],td)
    # A,B = get_linearized_dynamics(z_ref[2, i], prev_deltas[np.min([ i + 1, len(prev_deltas) - 1])],
    #                                             z_ref[3, i],td)
    constraints += [z[:,i+1] == A @ z[:,i] + B @ u[:,i]]

    #velocity limits
    # constraints += [z[3, i] <= v_max]
    # constraints += [z[3, i] >= v_min]

    #input limits
    constraints += [a_min <= u[1, i]]
    constraints += [u[1, i] <= a_max]
    constraints += [u[0, i] <= delta_f_max]
    constraints += [u[0, i] >= -delta_f_max]

    #rate constraints
    if i != 0:
        constraints += [(u[1, i] - u[1, i-1])/td <= a_rate_max]
        constraints += [(u[1, i] - u[1, i-1])/td >= -a_rate_max]
        constraints += [(u[0, i] - u[0, i-1])/td <= steer_rate]
        constraints += [(u[0, i] - u[0, i-1])/td >= -steer_rate]
    # else:
    #     constraints += [(u[1, i] - prev_accelerations[0])/ts  <= a_rate_max]
    #     constraints += [(u[1, i] - prev_accelerations[0]/ts ) >= -a_rate_max]
    #     constraints += [(u[0, i] - prev_deltas[0])/ts <= steer_rate]
    #     constraints += [(u[0, i] - prev_deltas[0])/ts >= -steer_rate]
#Quad program


In [67]:
qp = cvxpy.Problem(cvxpy.Minimize(cost), constraints)
qp.solve(solver=cvxpy.ECOS_BB, verbose=False)
print(qp.status)
if qp.status == cvxpy.OPTIMAL or qp.status == cvxpy.OPTIMAL_INACCURATE:
    x = np.array(z.value[0, :]).flatten()
    y = np.array(z.value[1, :]).flatten()
    v = np.array(z.value[3, :]).flatten()
    yaw = np.array(z.value[2, :]).flatten()
    a = np.array(u.value[1, :]).flatten()
    delta = np.array(u.value[0, :]).flatten()
else:
    a, delta = None, None

optimal


In [68]:
a

array([0.9999999 , 0.99999979, 0.99999921, 0.70000454, 0.40009036,
       0.        ])

In [69]:
delta

array([-0.35991425, -0.32491656, -0.28991786, -0.25491988, -0.21994115,
        0.        ])

In [70]:
z_ref

array([[ -16.84226799,  -14.93481376,  -13.57080091,  -13.38994163,
         -15.14255852,  -18.50294038],
       [-223.22581482, -217.56050298, -211.65647297, -205.58967784,
        -199.99066528, -194.93976997],
       [   1.22951789,    1.27888495,    1.42329674,    1.68506615,
           2.05180325,    2.2261284 ],
       [  10.        ,    8.        ,    5.        ,    4.        ,
           2.        ,    0.        ]])

In [71]:
x

array([-16.84226799, -17.26488875, -16.96397239, -17.07857254,
       -17.17339749, -17.23270762])

In [72]:
y

array([-223.22581482, -222.8585676 , -222.6614671 , -222.84967616,
       -222.95946649, -222.82987149])

In [73]:
v

array([13.22850092, 13.22464498, 13.22638926, 13.23241908, 13.24276288,
       13.25292491])